<a href="https://colab.research.google.com/github/maryamyazdi/transc/blob/text-classification/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install -q transformers

In [49]:
import numpy as np
import pandas as pd
from sklearn import metrics
import nltk
import re
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [69]:
# Setting up GPU usage (if available)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [70]:
init_df = pd.read_csv('stories.csv')
init_df

,body,topic
0,,['39822b5f-e37e-43e8-b997-7142fe55c3ea']
1,,['0d817400-3f5d-41e0-929c-c31fdbe75d31']
2,,['83a09c6b-5f2f-421f-ae50-b38acca7e008']
3,,['6fbf954a-03f9-4782-a65f-783271c9c447']
4,hello and welcome to BBC News a woman who gave...,"['83a09c6b-5f2f-421f-ae50-b38acca7e008', '9ff5..."
...,...,...
5176,News. More local help will soon be on the way....,"['9ff54ded-904b-4e0c-85ce-a3617f5cb913', '9632..."
5177,"with March 1, we start what is called Meteorol...",['9a06646a-e1df-4fca-888e-69658420556b']
5178,overseas. A massive Russian convoy is headed t...,['9ff54ded-904b-4e0c-85ce-a3617f5cb913']
5179,"And this morning, the National Hockey League s...","['9ff54ded-904b-4e0c-85ce-a3617f5cb913', 'b492..."


In [71]:
# Pre-processing the domain data (initial dataset)

nltk.download('stopwords')
from nltk.corpus import stopwords

df = init_df.copy()

# Remove empty body rows
df = df[df['body'].str.strip().astype(bool)]
df.reset_index(drop = True, inplace = True)

# Removing stop words
stop = stopwords.words('english')
df['body'] = df['body'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stop)]))

# Reformat 'topic' column to binary vectors
unique_topics = {}
a = set()
for index in df.index:
  df['topic'][index] = re.findall("[a-zA-Z0-9-]+",df['topic'][index])
  for id in df['topic'][index]:
    a.add(id)

unique_topics = {topic: index for (index,topic) in enumerate(a)}
count = {x:0 for x in unique_topics.keys()}

for index in df.index:
  e = [0] * len(unique_topics)
  for id in df.topic[index]:
    e[unique_topics[id]]=1
    count[id] +=1 
  df['topic'][index] = e   

df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,body,topic
0,hello welcome bbc news woman gave key evidence...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
1,news north hollywood. 14 yearold girl found de...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,homelessness city's greatest failure. message ...,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]"
3,minneapolis police officer kim potter guilty d...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,judy update wildfires wiped entire neighborhoo...,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
5147,news. local help soon way. group volunteers yo...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]"
5148,"march 1, start called meteorological spring. k...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
5149,overseas. massive russian convoy headed toward...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5150,"morning, national hockey league says suspendin...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]"


In [72]:
unique_topics

{'39822b5f-e37e-43e8-b997-7142fe55c3ea': 0,
 '0d817400-3f5d-41e0-929c-c31fdbe75d31': 1,
 'aa1edc37-1a01-414a-bcf7-8517e7c7053d': 2,
 '6fbf954a-03f9-4782-a65f-783271c9c447': 3,
 'e7cbe38d-c987-4113-aa94-fd77eda451d5': 4,
 '9ff54ded-904b-4e0c-85ce-a3617f5cb913': 5,
 'ca197b81-ca86-4792-8c25-2ba7cd4195b5': 6,
 '9a06646a-e1df-4fca-888e-69658420556b': 7,
 '83a09c6b-5f2f-421f-ae50-b38acca7e008': 8,
 '74e2fab8-689f-4e17-9a1c-e1f92e084f55': 9,
 'b49207eb-96eb-4b73-b534-adc0ef85022a': 10,
 'a58b4b70-1b59-4240-917d-a2165a0ce2f0': 11,
 'f5cdd7f2-9d4d-4ba5-9925-00c1701e30fa': 12,
 '96326734-fd82-4350-b45c-513e7eb9147c': 13,
 'ebf2991e-4b7d-44c6-927b-a261a7b21d2c': 14}

In [73]:
count

{'39822b5f-e37e-43e8-b997-7142fe55c3ea': 391,
 '0d817400-3f5d-41e0-929c-c31fdbe75d31': 183,
 'aa1edc37-1a01-414a-bcf7-8517e7c7053d': 163,
 '6fbf954a-03f9-4782-a65f-783271c9c447': 559,
 'e7cbe38d-c987-4113-aa94-fd77eda451d5': 22,
 '9ff54ded-904b-4e0c-85ce-a3617f5cb913': 1825,
 'ca197b81-ca86-4792-8c25-2ba7cd4195b5': 250,
 '9a06646a-e1df-4fca-888e-69658420556b': 409,
 '83a09c6b-5f2f-421f-ae50-b38acca7e008': 1793,
 '74e2fab8-689f-4e17-9a1c-e1f92e084f55': 253,
 'b49207eb-96eb-4b73-b534-adc0ef85022a': 388,
 'a58b4b70-1b59-4240-917d-a2165a0ce2f0': 53,
 'f5cdd7f2-9d4d-4ba5-9925-00c1701e30fa': 92,
 '96326734-fd82-4350-b45c-513e7eb9147c': 1076,
 'ebf2991e-4b7d-44c6-927b-a261a7b21d2c': 278}

In [74]:
# pd.DataFrame(count.items(), columns=['topic', 'frequency']);

In [75]:
# Defining some key variables for training stage

MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [76]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.body = dataframe.body
        self.targets = self.data.topic
        self.max_len = max_len

    def __len__(self):
        return len(self.body)

    def __getitem__(self, index):
        body = str(self.body[index])
        body = " ".join(body.split())

        inputs = self.tokenizer.encode_plus(
            body,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            # construct a tensor, then remove dimensions of size 1 with squeeze
            'ids': torch.squeeze(inputs['input_ids'].clone().detach()),
            'mask': torch.squeeze(inputs['attention_mask'].clone().detach()),
            'token_type_ids': torch.squeeze(inputs['token_type_ids'].clone().detach()),
            'targets': torch.squeeze(torch.tensor(self.targets[index], dtype=torch.float))
        }

In [77]:
train_size = 0.8
train_df = df.sample(frac=train_size, random_state=200).reset_index(drop=True)
test_df = df.drop(train_df.index).reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

# Each element in trainig_set or test_set is a dictionary consists of 4 tensors

FULL Dataset: (5152, 2)
TRAIN Dataset: (4122, 2)
TEST Dataset: (1030, 2)


In [78]:
training_set[3]['ids'].size()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


torch.Size([256])

In [79]:
# DataLoader

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 8
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'num_workers': 8
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [80]:
# Creating the network

class BERTClass(torch.nn.Module):
    def __init__(self, n_classes: int):
        super(BERTClass, self).__init__()
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased',return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, n_classes)
    
    def forward(self, ids, mask, token_type_ids):
        bert_output= self.bert(input_ids = ids, attention_mask = mask, token_type_ids = token_type_ids)
        dropout_output = self.dropout(bert_output.pooler_output)
        output = self.classifier(dropout_output)
        return output

# linear layer with 15 dimensions due to 15 different topic categories
model = BERTClass(n_classes=15)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [81]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [82]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [83]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [84]:
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, Loss:  0.7346986532211304
Epoch: 1, Loss:  0.24991366267204285
Epoch: 2, Loss:  0.1672009825706482
